In [8]:
import keras
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Activation
from keras.layers import Dense,Input,Conv2D,MaxPooling2D,UpSampling2D,Flatten,Reshape
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pn
from sklearn.model_selection import train_test_split

In [9]:
from keras.datasets import mnist
#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#X_train[33]

In [10]:
max_value=float(X_train.max())
X_train=X_train.astype('float32')/max_value
X_test=X_test.astype('float32')/max_value
max_value

255.0

In [11]:
X_train.shape

(60000, 28, 28)

In [12]:
X_test.shape

(10000, 28, 28)

In [13]:
X_train=X_train.reshape(len(X_train),28,28,1)
X_test=X_test.reshape(len(X_test),28,28,1)
X_train.shape

(60000, 28, 28, 1)

In [18]:
input_shape=X_train.shape[1:]
input_shape

(28, 28, 1)

In [22]:
autoencoder=Sequential()
#Encoder Layers
autoencoder.add(Conv2D(16,(3,3),activation='relu',padding='same',input_shape=X_train.shape[1:]))
autoencoder.add(MaxPooling2D((2,2),padding='same'))
autoencoder.add(Conv2D(8,(3,3),activation='relu',padding='same'))
autoencoder.add(MaxPooling2D((2,2),padding='same'))
autoencoder.add(Conv2D(8,(3,3),strides=(2,2),activation='relu',padding='same'))

#Flateen encoding for visualization
autoencoder.add(Flatten())
autoencoder.add(Reshape((4,4,8)))

#Decoder Layers
autoencoder.add(Conv2D(8,(3,3),activation='relu',padding='same'))
autoencoder.add(UpSampling2D((2,2)))
autoencoder.add(Conv2D(8,(3,3),activation='relu',padding='same'))
autoencoder.add(UpSampling2D((2,2)))
autoencoder.add(Conv2D(16,(3,3),activation='relu'))
autoencoder.add(UpSampling2D((2,2)))
autoencoder.add(Conv2D(1,(3,3),activation='sigmoid',padding='same'))

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 8)           584       
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 8)           0         
__________

In [29]:
encoder=Model(inputs=autoencoder.input,outputs=autoencoder.get_layer('flatten_3').output)
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 8)           584       
_________________________________________________________________
flatten_3 (Flatten)          (None, 128)               0         
Total para

In [30]:
autoencoder.compile(optimizer='adam',loss='binary_crossentropy')
autoencoder.fit(X_train,X_train,epochs=50,batch_size=256,shuffle=True,validation_data=(X_test,X_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 154s 3ms/step - loss: 0.2708 - val_loss: 0.1602
Epoch 2/50
60000/60000 [==============================] - 176s 3ms/step - loss: 0.1460 - val_loss: 0.1338
Epoch 3/50
60000/60000 [==============================] - 148s 2ms/step - loss: 0.1287 - val_loss: 0.1223
Epoch 4/50
60000/60000 [==============================] - 149s 2ms/step - loss: 0.1201 - val_loss: 0.1165
Epoch 5/50
60000/60000 [==============================] - 149s 2ms/step - loss: 0.1147 - val_loss: 0.1116
Epoch 6/50
60000/60000 [==============================] - 147s 2ms/step - loss: 0.1109 - val_loss: 0.1085
Epoch 7/50
60000/60000 [==============================] - 147s 2ms/step - loss: 0.1084 - val_loss: 0.1060
Epoch 8/50
60000/60000 [==============================] - 147s 2ms/step - loss: 0.1063 - val_loss: 0.1041
Epoch 9/50
60000/60000 [==============================] - 147s 2ms/step - loss: 0.1046 - val_loss: 0.1